In [27]:
import pandas as pd
import re
from datetime import datetime

In [28]:
odeme = pd.read_excel("0511.xlsx", sheet_name="odeme", dtype={"kwota": float})
bank = pd.read_excel("0511.xlsx", sheet_name="bank", dtype={"kwota": float})

In [29]:
# mapping is in odeme.xlsx, sheet faktury_mapping, columns: nr and nr zew, should be nr: nr zew
# faktury_mapping = pd.read_excel("odeme.xlsx", sheet_name="faktury_mapping")
# faktury_mapping = faktury_mapping.set_index("nr")
# faktury_mapping = faktury_mapping.to_dict()["nr zew"]

In [ ]:
def normalize_polish_characters(value):
    """Replaces Polish diacritical characters with their ASCII equivalents."""
    polish_to_ascii = {
        "Ą": "A", "Ć": "C", "Ę": "E", "Ł": "L", "Ń": "N",
        "Ó": "O", "Ś": "S", "Ź": "Z", "Ż": "Z",
        "ą": "a", "ć": "c", "ę": "e", "ł": "l", "ń": "n",
        "ó": "o", "ś": "s", "ź": "z", "ż": "z"
    }
    if isinstance(value, str):
        for char, ascii_char in polish_to_ascii.items():
            value = value.replace(char, ascii_char)
    return value

In [31]:
def find_res(tytul):
    """Extracts the first 7-digit number found in the string."""
    match = re.search(r"\d{7}", str(tytul))
    return match.group(0) if match else None

In [32]:
def to_date_string(value):
    try:
        # Check if the value is a float or integer (potential Excel numeric date)
        if isinstance(value, (int, float)):
            # Convert using Excel's 1900 date system (adjust for offset)
            base_date = datetime(1899, 12, 30)  # Excel's zero date (adjusted by 2 days)
            converted_date = base_date + pd.to_timedelta(value, unit='D')
            return converted_date.strftime("%d/%m/%Y")
        else:
            # Try to parse as a regular datetime string
            date = pd.to_datetime(value, errors='coerce')
            if pd.notnull(date):
                return date.strftime("%d/%m/%Y")
    except Exception:
        return value
    return value

In [33]:
odeme['kwota'] = odeme['kwota'].round(2) * 100
odeme['kwota'] = odeme['kwota'].astype(int)

bank['kwota'] = bank['kwota'].round(2) * 100
bank['kwota'] = bank['kwota'].astype(int)

In [34]:
odeme

,kwota,tytul,rez
0,55350,FSF/850/24/10/0003,COMMISSION FOR RES :1445490
1,65000,FSF/850/24/10/0003,COMMISSION FOR RES :1438694
2,137000,FSF/1005/24/10/0003,COMMISSION FOR RES :1254658
3,141000,FSF/1005/24/10/0003,COMMISSION FOR RES :1254660
4,32000,FSF/1005/24/10/0003,COMMISSION FOR RES :1402128
...,...,...,...
2294,71550,44/T/2024,Payment for res: 1444255
2295,71550,44/T/2024,Payment for res: 1443667
2296,124000,70/B/10/2024,Payment for res: 1289104
2297,67500,142/10/2024,Payment for res: 1452109


In [35]:
bank

,tytul,kwota
0,03456/10/24/P,500
1,00041/10/24/BP,671
2,02300/10/24/BP,1500
3,02020/10/24/BP,1500
4,02211/10/24/BP,2000
...,...,...
2056,10352/09/24/BP,838000
2057,ZWROT NADPŁATY Z REZ. 1460655,871604
2058,45570,876200
2059,335/2024,1196300


In [36]:
matched_lines = pd.DataFrame(columns=["Sum of Amounts", "List of Amounts", "Title", "List of REZ"])
unmatched_odeme = odeme.copy()
unmatched_bank = bank.copy()


In [37]:

# Find exact matches
for _, row in bank.iterrows():
    bank_kwota = row["kwota"]
    bank_tytul = row["tytul"]

    exact_matches = odeme[(odeme["kwota"] == bank_kwota) & (odeme["tytul"] == bank_tytul)]
    if not exact_matches.empty:
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [bank_kwota/100],
                "List of Amounts": [[bank_kwota/100]],
                "Title": [bank_tytul],
                "List of REZ": [exact_matches["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(exact_matches.index[0], axis=0)
        unmatched_bank = unmatched_bank.drop(row.name, axis=0)
        continue

# Find matches based on sums
remaining_bank = unmatched_bank.copy()


C:\Users\Kamil Tomaszek\AppData\Local\Temp\ipykernel_29140\3348580572.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matched_lines = pd.concat([


In [38]:

for _, row in remaining_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_tytul = row["tytul"]

    matching_odeme = unmatched_odeme[unmatched_odeme["tytul"] == bank_tytul]
    if not matching_odeme.empty:
        sum_odeme = matching_odeme["kwota"].sum()

        if sum_odeme == bank_kwota:
            matched_lines = pd.concat([
                matched_lines,
                pd.DataFrame({
                    "Sum of Amounts": [sum_odeme/100],
                    "List of Amounts": [[amount / 100 for amount in matching_odeme["kwota"]]],
                    "Title": [bank_tytul],
                    "List of REZ": [matching_odeme["rez"].tolist()]
                })
            ], ignore_index=True)
            unmatched_odeme = unmatched_odeme.drop(matching_odeme.index, axis=0)
            unmatched_bank = unmatched_bank.drop(row.name, axis=0)


In [39]:
# Match based on kwota and find_res applied to tytul
unmatched_odeme["res_group"] = unmatched_odeme["tytul"].apply(find_res)
unmatched_odeme["res_group2"] = unmatched_odeme["rez"].apply(find_res)
unmatched_bank["res_group"] = unmatched_bank["tytul"].apply(find_res)

for _, row in unmatched_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_res = row["res_group"]

    # Match based on kwota and extracted res_group
    partial_matches = unmatched_odeme[
        (unmatched_odeme["kwota"] == bank_kwota) & (unmatched_odeme["res_group"] == bank_res)
    ]
    if not partial_matches.empty:
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [bank_kwota / 100],
                "List of Amounts": [[amount / 100 for amount in partial_matches["kwota"]]],
                "Title": [row["tytul"]],  # Keep the original tytul
                "List of REZ": [partial_matches["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(partial_matches.index, axis=0)
        unmatched_bank = unmatched_bank.drop(row.name, axis=0)


In [40]:
unmatched_odeme["res_group2"] = unmatched_odeme["rez"].apply(find_res)  # Create a new res_group2 based on find_res

for _, row in unmatched_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_res = row["res_group"]

    # Match based on kwota and res_group2
    secondary_matches = unmatched_odeme[
        (unmatched_odeme["kwota"] == bank_kwota) & (unmatched_odeme["res_group2"] == bank_res)
    ]
    if not secondary_matches.empty:
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [bank_kwota / 100],
                "List of Amounts": [[amount / 100 for amount in secondary_matches["kwota"]]],
                "Title": [row["tytul"]],  # Keep the original tytul
                "List of REZ": [secondary_matches["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(secondary_matches.index, axis=0)
        unmatched_bank = unmatched_bank.drop(row.name, axis=0)

In [41]:
# Normalize dates in unmatched_odeme and unmatched_bank
unmatched_odeme["normalized_date"] = unmatched_odeme["tytul"].apply(to_date_string)
unmatched_bank["normalized_date"] = unmatched_bank["tytul"].apply(to_date_string)

# Compare based on kwota and normalized_date
for _, row in unmatched_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_date = row["normalized_date"]

    # Check for direct matches (1-to-1)
    direct_matches = unmatched_odeme[
        (unmatched_odeme["kwota"] == bank_kwota) & (unmatched_odeme["normalized_date"] == bank_date)
    ]

    if not direct_matches.empty:
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [bank_kwota / 100],
                "List of Amounts": [[amount / 100 for amount in direct_matches["kwota"]]],
                "Title": [row["tytul"]],  # Keep the original tytul
                "List of REZ": [direct_matches["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(direct_matches.index, axis=0)
        unmatched_bank = unmatched_bank.drop(row.name, axis=0)
        continue

    # Check for 1-to-many matches (sum of kwota)
    matching_odeme = unmatched_odeme[unmatched_odeme["normalized_date"] == bank_date]
    if not matching_odeme.empty:
        sum_odeme = matching_odeme["kwota"].sum()

        if sum_odeme == bank_kwota:
            matched_lines = pd.concat([
                matched_lines,
                pd.DataFrame({
                    "Sum of Amounts": [sum_odeme / 100],
                    "List of Amounts": [[amount / 100 for amount in matching_odeme["kwota"]]],
                    "Title": [row["tytul"]],  # Keep the original tytul
                    "List of REZ": [matching_odeme["rez"].tolist()]
                })
            ], ignore_index=True)
            unmatched_odeme = unmatched_odeme.drop(matching_odeme.index, axis=0)
            unmatched_bank = unmatched_bank.drop(row.name, axis=0)


C:\Users\Kamil Tomaszek\AppData\Local\Temp\ipykernel_29140\706849819.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  date = pd.to_datetime(value, errors='coerce')


In [42]:
matched_lines["Title2"] = matched_lines["Title"]
matched_lines = matched_lines[["Sum of Amounts", "List of Amounts", "Title", "Title2", "List of REZ"]]

In [43]:
matched_lines

,Sum of Amounts,List of Amounts,Title,Title2,List of REZ
0,5.00,[5.0],03456/10/24/P,03456/10/24/P,[COMMISSION FOR RES :1343024]
1,6.71,[6.71],00041/10/24/BP,00041/10/24/BP,[COMMISSION FOR RES :1042120]
2,15.00,[15.0],02300/10/24/BP,02300/10/24/BP,[COMMISSION FOR RES :1433885]
3,15.00,[15.0],02020/10/24/BP,02020/10/24/BP,[COMMISSION FOR RES :1260317]
4,20.00,[20.0],02211/10/24/BP,02211/10/24/BP,[COMMISSION FOR RES :1423680]
...,...,...,...,...,...
2038,504.00,[504.0],Fv. KOSZT UGODA Z KLIENTEM RES.1383035 Z DN 2...,Fv. KOSZT UGODA Z KLIENTEM RES.1383035 Z DN 2...,[THE COST OF THE SETTLEMENT WITH THE CLIENT OF...
2039,506.22,[506.22],Fv. KOSZT UGODA Z KLIENTEM RES.1255885 Z DN 2...,Fv. KOSZT UGODA Z KLIENTEM RES.1255885 Z DN 2...,[THE COST OF THE SETTLEMENT WITH THE CLIENT OF...
2040,520.00,[520.0],Fv. KOSZT UGODA Z KLIENTEM RES. 1397591 Z DN ...,Fv. KOSZT UGODA Z KLIENTEM RES. 1397591 Z DN ...,[THE COST OF THE SETTLEMENT WITH THE CLIENT OF...
2041,1290.00,[1290.0],45591,45591,[Payment for res: 1205283]


In [44]:
# Apply faktury_mapping to tytul in odeme and bank
faktury_mapping = {}
unmatched_odeme["mapped_tytul"] = unmatched_odeme["tytul"].map(faktury_mapping).fillna(unmatched_odeme["tytul"]).apply(to_date_string)
unmatched_bank["mapped_tytul"] = unmatched_bank["tytul"].map(faktury_mapping).fillna(unmatched_bank["tytul"]).apply(to_date_string)

# Compare based on kwota and mapped_tytul (with original and mapped names)
for _, row in unmatched_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_mapped_tytul = row["mapped_tytul"]

    # Check for direct matches (1-to-1)
    direct_matches = unmatched_odeme[
        (unmatched_odeme["kwota"] == bank_kwota) & (unmatched_odeme["mapped_tytul"] == bank_mapped_tytul)
    ]

    if not direct_matches.empty:
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [bank_kwota / 100],
                "List of Amounts": [[amount / 100 for amount in direct_matches["kwota"]]],
                "Title": direct_matches["tytul"].tolist(),
                "Title2": [bank_mapped_tytul],
                "List of REZ": [direct_matches["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(direct_matches.index, axis=0)
        unmatched_bank = unmatched_bank.drop(row.name, axis=0)
        continue

    # Check for 1-to-many matches (sum of kwota)
    matching_odeme = unmatched_odeme[unmatched_odeme["mapped_tytul"] == bank_mapped_tytul]
    if not matching_odeme.empty:
        sum_odeme = matching_odeme["kwota"].sum()

    if sum_odeme == bank_kwota:
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [sum_odeme / 100],
                "List of Amounts": [[amount / 100 for amount in matching_odeme["kwota"]]],
                "Title": [matching_odeme["tytul"].tolist()],  # Wrap list in another list for consistency
                "Title2": [[bank_mapped_tytul] * len(matching_odeme)],  # Repeat the mapped title for each match
                "List of REZ": [matching_odeme["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(matching_odeme.index, axis=0)
        unmatched_bank = unmatched_bank.drop(row.name, axis=0)


In [45]:
# Group unmatched_odeme and unmatched_bank by mapped_tytul and sum kwota
odeme_grouped = unmatched_odeme.groupby("mapped_tytul")["kwota"].sum().reset_index()
bank_grouped = unmatched_bank.groupby("mapped_tytul")["kwota"].sum().reset_index()

# Merge the grouped data on mapped_tytul to compare sums
merged_groups = pd.merge(odeme_grouped, bank_grouped, on="mapped_tytul", suffixes=("_odeme", "_bank"))

# Iterate through the merged groups and find matches
for _, group_row in merged_groups.iterrows():
    mapped_tytul = group_row["mapped_tytul"]
    odeme_sum = group_row["kwota_odeme"]
    bank_sum = group_row["kwota_bank"]

    # Check if the sums match
    if odeme_sum == bank_sum:
        # Get the matching rows from unmatched_odeme and unmatched_bank
        matching_odeme = unmatched_odeme[unmatched_odeme["mapped_tytul"] == mapped_tytul]
        matching_bank = unmatched_bank[unmatched_bank["mapped_tytul"] == mapped_tytul]

        # Add to matched_lines
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [odeme_sum / 100],
                "List of Amounts": [[
                    [amount / 100 for amount in matching_odeme["kwota"]],  # List from odeme
                    [amount / 100 for amount in matching_bank["kwota"]]   # List from bank
                ]],
                "Title": [matching_odeme["tytul"].tolist()],  # Original titles from odeme
                "Title2": [matching_bank["tytul"].tolist()],  # Original titles from bank
                "List of REZ": [matching_odeme["rez"].tolist()]  # List of rez from odeme
            })
        ], ignore_index=True)

        # Drop matched rows from unmatched_odeme and unmatched_bank
        unmatched_odeme = unmatched_odeme.drop(matching_odeme.index, axis=0)
        unmatched_bank = unmatched_bank.drop(matching_bank.index, axis=0)


In [ ]:

# Function to switch mm/dd/yyyy to dd/mm/yyyy
def switch_date_format(value):
    # Ensure value is a string
    value_str = str(value)
    match = re.match(r"^(\d{2})/(\d{2})/(\d{4})$", value_str)
    if match:
        # Swap day and month
        return f"{match.group(2)}/{match.group(1)}/{match.group(3)}"
    # Return the original value if no match
    return value


# Apply the function to odeme[mapped_tytul]
unmatched_odeme["mapped_tytul"] = unmatched_odeme["mapped_tytul"].apply(switch_date_format)

# Step 1: One-to-One Comparison
for _, row in unmatched_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_mapped_tytul = row["mapped_tytul"]

    direct_matches = unmatched_odeme[
        (unmatched_odeme["kwota"] == bank_kwota) & (unmatched_odeme["mapped_tytul"] == bank_mapped_tytul)
    ]

    if not direct_matches.empty:
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [bank_kwota / 100],
                "List of Amounts": [[amount / 100 for amount in direct_matches["kwota"]]],
                "Title": direct_matches["tytul"].tolist(),
                "Title2": [bank_mapped_tytul],
                "List of REZ": [direct_matches["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(direct_matches.index, axis=0)
        unmatched_bank = unmatched_bank.drop(row.name, axis=0)

# Step 2: One-to-Many Comparison
for _, row in unmatched_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_mapped_tytul = row["mapped_tytul"]

    # Find all unmatched odeme rows with the same mapped_tytul
    matching_odeme = unmatched_odeme[unmatched_odeme["mapped_tytul"] == bank_mapped_tytul]
    if not matching_odeme.empty:
        # Calculate the sum of kwota for the matching odeme rows
        sum_odeme = matching_odeme["kwota"].sum()

        # Check if the sum matches the bank_kwota
        if sum_odeme == bank_kwota:
            matched_lines = pd.concat([
                matched_lines,
                pd.DataFrame({
                    "Sum of Amounts": [sum_odeme / 100],
                    "List of Amounts": [[
                        [amount / 100 for amount in matching_odeme["kwota"].tolist()],  # List from odeme
                        [bank_kwota / 100]  # Single amount from bank
                    ]],
                    "Title": [matching_odeme["tytul"].tolist()],  # Original titles from odeme
                    "Title2": [[bank_mapped_tytul] * len(matching_odeme)],  # Mapped title repeated for each odeme row
                    "List of REZ": [matching_odeme["rez"].tolist()]  # List of rez from odeme
                })
            ], ignore_index=True)

            # Drop matched rows from unmatched_odeme and unmatched_bank
            unmatched_odeme = unmatched_odeme.drop(matching_odeme.index, axis=0)
            unmatched_bank = unmatched_bank.drop(row.name, axis=0)

# Step 3: Many-to-Many Comparison
odeme_grouped = unmatched_odeme.groupby("mapped_tytul")["kwota"].sum().reset_index()
bank_grouped = unmatched_bank.groupby("mapped_tytul")["kwota"].sum().reset_index()

merged_groups = pd.merge(odeme_grouped, bank_grouped, on="mapped_tytul", suffixes=("_odeme", "_bank"))

for _, group_row in merged_groups.iterrows():
    mapped_tytul = group_row["mapped_tytul"]
    odeme_sum = group_row["kwota_odeme"]
    bank_sum = group_row["kwota_bank"]

    if odeme_sum == bank_sum:
        matching_odeme = unmatched_odeme[unmatched_odeme["mapped_tytul"] == mapped_tytul]
        matching_bank = unmatched_bank[unmatched_bank["mapped_tytul"] == mapped_tytul]

        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [odeme_sum / 100],
                "List of Amounts": [[
                    [amount / 100 for amount in matching_odeme["kwota"].tolist()],
                    [amount / 100 for amount in matching_bank["kwota"]]
                ]],
                "Title": matching_odeme["tytul"].tolist(),
                "Title2": matching_bank["tytul"].tolist(),
                "List of REZ": [matching_odeme["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(matching_odeme.index, axis=0)
        unmatched_bank = unmatched_bank.drop(matching_bank.index, axis=0)


In [48]:
# Normalize mapped_tytul in unmatched_odeme and unmatched_bank
unmatched_odeme["normalized_tytul"] = unmatched_odeme["mapped_tytul"].apply(normalize_polish_characters)
unmatched_bank["normalized_tytul"] = unmatched_bank["mapped_tytul"].apply(normalize_polish_characters)


In [49]:
for _, row in unmatched_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_normalized_tytul = row["normalized_tytul"]

    # Find direct matches in odeme
    direct_matches = unmatched_odeme[
        (unmatched_odeme["kwota"] == bank_kwota) & (unmatched_odeme["normalized_tytul"] == bank_normalized_tytul)
    ]

    if not direct_matches.empty:
        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [bank_kwota / 100],
                "List of Amounts": [[amount / 100 for amount in direct_matches["kwota"]]],
                "Title": direct_matches["tytul"].tolist(),
                "Title2": [row["mapped_tytul"]],
                "List of REZ": [direct_matches["rez"].tolist()]
            })
        ], ignore_index=True)
        unmatched_odeme = unmatched_odeme.drop(direct_matches.index, axis=0)
        unmatched_bank = unmatched_bank.drop(row.name, axis=0)

# 2. One-to-Many Comparison
for _, row in unmatched_bank.iterrows():
    bank_kwota = row["kwota"]
    bank_normalized_tytul = row["normalized_tytul"]

    # Find unmatched odeme rows with the same normalized_tytul
    matching_odeme = unmatched_odeme[unmatched_odeme["normalized_tytul"] == bank_normalized_tytul]
    if not matching_odeme.empty:
        # Calculate the sum of kwota for the matching odeme rows
        sum_odeme = matching_odeme["kwota"].sum()

        # Check if the sum matches the bank_kwota
        if sum_odeme == bank_kwota:
            matched_lines = pd.concat([
                matched_lines,
                pd.DataFrame({
                    "Sum of Amounts": [sum_odeme / 100],
                    "List of Amounts": [[
                        [amount / 100 for amount in matching_odeme["kwota"].tolist()],
                        [bank_kwota / 100]
                    ]],
                    "Title": [matching_odeme["tytul"].tolist()],
                    "Title2": [[row["mapped_tytul"]] * len(matching_odeme)],
                    "List of REZ": [matching_odeme["rez"].tolist()]
                })
            ], ignore_index=True)

            # Drop matched rows from unmatched_odeme and unmatched_bank
            unmatched_odeme = unmatched_odeme.drop(matching_odeme.index, axis=0)
            unmatched_bank = unmatched_bank.drop(row.name, axis=0)

# 3. Many-to-Many Comparison
odeme_grouped = unmatched_odeme.groupby("normalized_tytul")["kwota"].sum().reset_index()
bank_grouped = unmatched_bank.groupby("normalized_tytul")["kwota"].sum().reset_index()

# Merge the grouped data on normalized_tytul to compare sums
merged_groups = pd.merge(odeme_grouped, bank_grouped, on="normalized_tytul", suffixes=("_odeme", "_bank"))

for _, group_row in merged_groups.iterrows():
    normalized_tytul = group_row["normalized_tytul"]
    odeme_sum = group_row["kwota_odeme"]
    bank_sum = group_row["kwota_bank"]

    # Check if the sums match
    if odeme_sum == bank_sum:
        # Get the matching rows from unmatched_odeme and unmatched_bank
        matching_odeme = unmatched_odeme[unmatched_odeme["normalized_tytul"] == normalized_tytul]
        matching_bank = unmatched_bank[unmatched_bank["normalized_tytul"] == normalized_tytul]

        matched_lines = pd.concat([
            matched_lines,
            pd.DataFrame({
                "Sum of Amounts": [odeme_sum / 100],
                "List of Amounts": [[
                    [amount / 100 for amount in matching_odeme["kwota"].tolist()],
                    [amount / 100 for amount in matching_bank["kwota"].tolist()]
                ]],
                "Title": matching_odeme["tytul"].tolist(),
                "Title2": matching_bank["tytul"].tolist(),
                "List of REZ": [matching_odeme["rez"].tolist()]
            })
        ], ignore_index=True)

        # Drop matched rows from unmatched_odeme and unmatched_bank
        unmatched_odeme = unmatched_odeme.drop(matching_odeme.index, axis=0)
        unmatched_bank = unmatched_bank.drop(matching_bank.index, axis=0)

In [50]:
unmatched_odeme['kwota'] = unmatched_odeme['kwota'].astype(float) / 100
unmatched_bank['kwota'] = unmatched_bank['kwota'].astype(float) / 100

In [51]:

# Save results to an Excel file
output_path = "wynik0511.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    matched_lines.to_excel(writer, sheet_name="Matched Lines", index=False)
    unmatched_odeme.to_excel(writer, sheet_name="Unmatched Odeme", index=False)
    unmatched_bank.to_excel(writer, sheet_name="Unmatched Bank", index=False)

output_path

'wynik0511.xlsx'